# Lab 3 - Computational Intelligence 2024

In [79]:
from collections import namedtuple, deque
from queue import *
from random import *
from tqdm.auto import tqdm
import numpy as np
from time import time


In [ ]:
PUZZLE_DIM = 3
RANDOMIZE_STEPS = 100_000

action = namedtuple('Action', ('pos1', 'pos2'))

### Game Variant
_You can choose to pursuit your own goal.<br> Just set down here in_ ___GOAL_STATE___ _variable the sequence to reach._

In [48]:
#   Default game: search for number ordered increasing
default_game = True 

#   If we wanna search a Specific configuration, ENTER here the configuration following the rules:
#   -   MUST contain ONLY one "0";
#   -   write number from 1 to PUZZLE_DIM**2 - 1 in a list

GOAL_STATE = [1, 2, 3, 0]

if not default_game:
    if GOAL_STATE.count(0) != 1:
        print("You insered too many 0s in your goal state!\n", GOAL_STATE)
        exit(1)

    if len(set(GOAL_STATE)) != PUZZLE_DIM**2:
        print("You haven't created a feasible solution!\n",
              f"--> Remeber that the goal must be a {PUZZLE_DIM}x{PUZZLE_DIM} matrix,\n",
              f"    with all the number from 0 to {PUZZLE_DIM-1}!\n", GOAL_STATE)
        exit(1)

    GOAL_STATE = np.array(GOAL_STATE).reshape((PUZZLE_DIM, PUZZLE_DIM))
else:
    GOAL_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

GOAL_STATE

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 0]])

### Utility functions

In [49]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [51]:
def solution_check(state: np.ndarray) -> bool:
    return np.array_equal(state, GOAL_STATE)

In [52]:
def avoid_loop(state: np.ndarray, visited_state) -> bool:
    if all(isinstance(item, np.ndarray) for item in visited_state):
        return any(np.array_equal(state, s) for s in visited_state)
    else:
        return any(np.array_equal(state, s.board) for s in visited_state)


In [78]:
#   Random initialize the game board   #
INITIAL_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for _ in range(RANDOMIZE_STEPS):
    INITIAL_STATE = do_action(INITIAL_STATE, choice(available_actions(INITIAL_STATE)))
print(INITIAL_STATE)

[[3 6 4]
 [2 1 7]
 [0 5 8]]


### Randomized Solution

In [107]:
state = INITIAL_STATE

for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    #print(state)
    state = do_action(state, choice(available_actions(state)))
    if solution_check(state):
        break
print(f"Solved in {r:,} steps")

Randomizing:   0%|          | 11/100000 [00:00<?, ?it/s]

Solved in 11 steps


## PATH SEARCH STRATEGY

### Depth-First Strategy

In [118]:
frontier = deque([INITIAL_STATE])

explored_state = list()
#   action_sequence = list()
state = frontier.pop()
explored_state.append(state)
explored_node = 1

with tqdm(total=None, desc="Depth-First") as pbar:
    while not solution_check(state):
        print(state, "\n")
        for a in available_actions(state):
            new_state = do_action(state, a)
            if not avoid_loop(new_state, explored_state):
                frontier.appendleft(new_state)

        print(len(frontier))
        state = frontier.popleft()
        explored_state.append(state)
        explored_node += 1
        pbar.update(1)

print(f"Solved in {explored_node:,} steps")

Depth-First: 2it [00:00, ?it/s]

[[0 1]
 [3 2]] 

2
[[1 0]
 [3 2]] 

2
Solved in 3 steps


### Breadth-First Strategy

In [94]:
frontier = deque([INITIAL_STATE])

explored_state = list()
#   action_sequence = list()
state = frontier.pop()
explored_state.append(state)
explored_node = 1

with tqdm(total=None, desc="Depth-First") as pbar:
    
    while not solution_check(state):
        actions = available_actions(state)

        for a in actions:
            new_state = do_action(state, a)
            if not avoid_loop(new_state, explored_state):
                frontier.append(new_state)

        #print(len(frontier))
        state = frontier.popleft()
        explored_state.append(state)
        explored_node += 1
        pbar.update(1)

print(f"Solved in {explored_node:,} steps")

Depth-First: 7it [00:00, ?it/s]

Solved in 8 steps


### A* Strategy

In [53]:
def heuristic(state: np.ndarray) -> int:
    '''
        Use the classic distance definition, of each point from its correct position, as cost
    '''
    n = state.shape[0]
    distance = 0

    for (i, j), tile in np.ndenumerate(state):
        if tile == 0:
            continue

        target_i, target_j = divmod(tile-1, n)
        distance += abs(i - target_i) + abs(j - target_j)

    return distance

In [72]:
class Node:
    def __init__(self, board: np.ndarray, previous_state: "Node") -> None:
        self.board = board      # Game state
        self.previous_state = previous_state    # Parent Node
        if previous_state is None:      # Actual Cost to reach this Node
            self.g = 0
        else:
            self.g = previous_state.g + 1
        self.h = heuristic(board)
        self.f = (self.g) + (self.h)    # f(x) = g(x) + h(x)

    def __eq__(self, other: "Node"):
        return np.array_equal(self.board, other.board)

    def __lt__(self, other: "Node"):
        return (self.f, self.g) < (other.f, other.g)

    def __repr__(self):
        """
        Rappresentazione leggibile del nodo.
        """
        stato_str = '\n'.join([' '.join(map(str, riga)) for riga in self.board])
        return f"Move number: {self.g}\nBoard:\n{stato_str}\n"

In [55]:
def avoid_useless_state(node: "Node", frontier: PriorityQueue[Node]) -> bool:
    for existing_node in frontier:
        if existing_node == node :
            if existing_node < node:
                return True
        
    return False


In [ ]:
def A_star(prob):
    frontier = PriorityQueue()

    explored_state = list()
    state = Node(prob, None)
    explored_state.append(state)

    while not solution_check(state.board):
        for a in available_actions(state.board):
            new_node = Node(do_action(state.board, a), state)
            if not avoid_loop(new_node.board, explored_state):
                frontier.put(new_node)
        state = frontier.get()
        #print(state.h)
        explored_state.append(state)

    return explored_state

# Show Solution

In [ ]:
current_node = A_star(INITIAL_STATE)[-1]
path = []
while current_node:
    path.append(current_node)
    current_node = current_node.previous_state
print(f"Visited {len(explored_state):,} states. Problem can be solved in {len(path)-1} moves !")

path[::-1]  # Return reversed path from start to goal


## Benchmarking

In [83]:
def random_initialization(puzzleDimension: int) -> np.ndarray:
    #   Random initialize the game board   #
    INITIAL_STATE = np.array([i for i in range(1, puzzleDimension**2)] + [0]).reshape((puzzleDimension, puzzleDimension))
    for _ in range(RANDOMIZE_STEPS):
        INITIAL_STATE = do_action(INITIAL_STATE, choice(available_actions(INITIAL_STATE)))
    return INITIAL_STATE

In [ ]:
for PUZZLE_DIM in range(2, 8):
    print(f"Dimensions: {PUZZLE_DIM}x{PUZZLE_DIM} -> ", end="")
    prob = random_initialization(PUZZLE_DIM)
    start = time()
    sol = A_star(prob)
    end = time()
    print(f"Steps: {len(sol) - 1} ({(end - start):.3f}s)", end="")
    if PUZZLE_DIM < 4:
        sol = A_star(prob)
        print(f" (Optimal #steps: {len(sol) - 1})", end="")
    print()

Dimensions: 2x2 -> 1
1
2
2
3
3
4
4
5
5
6
6
